In [1]:
import os
from dotenv import load_dotenv
# ✅ Step 1: Load environment variables
load_dotenv()
# Make sure to set your OpenAI API key in the .env file or directly here

# ✅ Step 2: Import libraries
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain import hub
import openai
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel

import asyncio
import nest_asyncio
asyncio.set_event_loop_policy(asyncio.DefaultEventLoopPolicy())
nest_asyncio.apply()
import gradio as gr

openai.api_key = os.getenv("OPENAI_API_KEY")  # Ensure you have set this in your .env file
embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)
vectorstore = FAISS.load_local("nba_vector_db", embeddings=embedding, allow_dangerous_deserialization=True)

C:\Users\16343\AppData\Local\Temp\ipykernel_19288\268236888.py:25: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)


In [2]:
## Define the Retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})\
## Define the prompt
prompt = hub.pull("jclemens24/rag-prompt")

c:\Users\16343\anaconda3\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [3]:
relevance_prompt_template = PromptTemplate.from_template(
    """
    Given the following question and retrieved context, determine if the context is relevant to the question.
    Provide a score from 1 to 5, where 1 is not at all relevant and 5 is highly relevant.
    Return ONLY the numeric score, without any additional text or explanation.

    Question: {question}
    Retrieved Context: {retrieved_context}

    Relevance Score:"""
)

In [4]:
def format_docs(docs):
 return "\n\n".join(doc.page_content for doc in docs)

In [5]:
def extract_score(llm_output):
    try:
        score = float(llm_output.strip())
        return score
    except ValueError:
        return 0

# Chain it all together with LangChain
def conditional_answer(x):
    relevance_score = extract_score(x['relevance_score'])
    if relevance_score < 4:
        return "I don't know."
    else:
        return x['answer']

In [6]:
## Define the LLM
llm = ChatOpenAI(model_name="gpt-4o-mini")
str_output_parser = StrOutputParser()

In [7]:
## Define the chain
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | RunnableParallel(
        {"relevance_score": (
            RunnablePassthrough()
            | (lambda x: relevance_prompt_template.format(question=x['question'], retrieved_context=x['context']))
            | llm
            | str_output_parser
        ), "answer": (
            RunnablePassthrough()
            | prompt
            | llm
            | str_output_parser
        )}
    )
    | RunnablePassthrough().assign(final_answer=conditional_answer)
)

In [8]:
rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [11]:
result = rag_chain_with_source.invoke("Who is the GOAT of NBA?")
relevance_score = result['answer']['relevance_score']
final_answer = result['answer']['final_answer']

print(f"Relevance Score: {relevance_score}")
print(f"Final Answer:\n{final_answer}")

Relevance Score: 5
Final Answer:
The GOAT (Greatest of All Time) of the NBA is often considered to be Michael Jordan. Many of his contemporaries and analysts regard him as the greatest basketball player of all time, and polls typically show Jordan ranking above other players, such as LeBron James, who is also frequently mentioned in the GOAT debate.


In [14]:
# Gradio Interface
def process_question(question):
    result = rag_chain_with_source.invoke(question)
    relevance_score = result['answer']['relevance_score']
    final_answer = result['answer']['final_answer']
    return relevance_score, final_answer

demo = gr.Interface(
    fn=process_question,
    inputs=gr.Textbox(label="Enter your question"),
    outputs=[
        gr.Textbox(label="Relevance Score"),
        gr.Textbox(label="Final Answer")
    ],
    title="NBA Player Question Answering",
    description="Enter a question and get the relevance score, final answer, and sources from RAG."
)

In [15]:
demo.launch(share=True, debug=False, auth=("admin", "pass1234")) # with credentials

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://1ab4ed007142e86579.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
